In [1]:
using PyPlot

INFO: Loading help data...


## Problem in $R^n$

In this file, we study the convergence of the Gradient Descent and Newton's method for a larger problem. Feel free, to pick $n$ as you wish and see how this choice impacts the performance.
We consider the following minimization problem

$$
    \min_{x\in R^n}  - \sum_{i=1}^n \log(1 - x_i^2) - \sum_{i=1}^n \log(b_i - a_i^T x)
$$
where $a_i$ denotes the $i$th row of the sparse matrix $A$ with random coefficients and $b$ is drawn from a uniform distribution (having values in $[0,1]$).

In [3]:
# this code sets up the problem and derivatives.
n = 100
e = ones(Float64,n)
A = sprandn(n,n,3*n./n^2)
b = rand(n)

f(x) = ((minimum(1-x.^2)>=0) && (minimum(b - A*x)>=0)) ? - dot(e,log(1-x.^2)) - dot(e,log(b - A*x)) : Inf
df(x) = 2.*(x./(1-x.^2))+A'*(1./(b-A*x))  
d2f(x) =  2* spdiagm((x.^2 + 1)./(x.^2-1).^2,0,n,n) + A'*spdiagm(1./((b-A*x).^2),0,n,n)*A

d2f (generic function with 1 method)

## Newton's method

we now apply Newton's method (with fixed line search of $1$) to the problem. The function we call also is contained in ``optiFuncs.jl``. The function looks like
```
function newton(f::Function,df::Function,H::Function,x::Vector;maxIter=20,atol=1e-8,doPrint=false)
    .
    .
    .
    return x,his,X
end
```

In [ ]:
include("/home/juser/math346/optiFuncs.jl")

x0 = zeros(Float64,n)
xnt,hisnt = newton(f,df,d2f,x0,doPrint=false,maxIter=20,atol=1e-20);

subplot(1,2,1)
semilogy(hisnt[:,1]-hisnt[end,1])
title("suboptimality, f(xk) - p*")
xlabel("iteration")
subplot(1,2,2)
semilogy(hisnt[:,2])
title("norm of gradient, |df(x_k)|")
xlabel("iteration")

@printf "solution has function value %1.15f" f(xnt)

## Gradient Descent Method

The following code uses the function ``gd`` in the file ``optiFuncs.jl``. Make sure to include it. The function looks like this

```
function gd(f::Function,df::Function,x::Vector;maxIter=20,atol=1e-8,doPrint=false,ls::Function=armijo)
    .
    .
    .
    return x,his,X
end
```


In [1]:
myLS(f,fc,df,x,pk) =  armijo(f,fc,df,x,pk,alpha=1e-1,b=0.9,maxIter=2000)
x0 = zeros(Float64,n)
xgd,hisgd,Xgd = gd(f,df,x0,maxIter=100,doPrint=false,ls=myLS);

subplot(1,2,1)
semilogy(hisgd[:,1]-hisnt[end,1])
title("suboptimality, f(xk) - p*")
xlabel("iteration")
subplot(1,2,2)
semilogy(hisgd[:,2])
title("norm of gradient, |df(x_k)|")
xlabel("iteration")


LoadError: n not defined
while loading In[1], in expression starting on line 2